查看模型結構與MSE

In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


s_c = '結算價'
model_path = './../../../'
expiry = 'NearbyMonth'

type_ = 'call'
IV_type = '{}IV'.format(type_)
if type_ == 'call':
    K_Range = [300, 500]
if type_ == 'put':
    K_Range = [500, 300]

K_Range_file = '{}_{}.csv'.format(K_Range[0], K_Range[1])
K_Range_dir = '{}_{}'.format(K_Range[0], K_Range[1])

#model_type = 'LSTM-LSTM'
model_type = 'TimeLinear'
#model_type = 'CNN-LSTM'
#model_type = 'ANN'
seq_length1 = 15
seq_length2 = seq_length1+1
min_contract_day = 2
forecast_variable = 'slope'
model_name = 'model1'

model_Dir_tree = ['Forecast&model', expiry, IV_type, 'K_{}'.format(K_Range_dir), model_type,\
                  'seq{}_seq{}_min{}'.format(seq_length1, seq_length2, min_contract_day), forecast_variable]
for model_dir in model_Dir_tree:
    if model_dir not in os.listdir(model_path):
        os.mkdir(model_path + model_dir)
    model_path = model_path + model_dir + '/'

#ModelImg_path = model_path + '{}.png'.format(model_name)
#ModelImg = Image.open(ModelImg_path)


Modeltxt_path = model_path + '{}.txt'.format(model_name)

# 打开文件并逐行读取
with open(Modeltxt_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 逐行打印文件内容
for line in lines:
    print(line.strip())  # 使用strip()去除每行末尾的换行符

#plt.figure(figsize=(15, 15))
#plt.imshow(ModelImg)
#plt.axis('off')  # 关闭坐标轴
#plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './../../../Forecast&model/NearbyMonth/callIV/K_300_500/TimeLinear/seq15_seq16_min2/slope/model1.txt'

查看各項損失指標

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

K_num = 100
s_c = '結算價'
model_path = './../../../'
expiry = 'NearbyMonth'

type_ = 'call'
IV_type = '{}IV'.format(type_)
if type_ == 'call':
    K_Range = [300, 500]
if type_ == 'put':
    K_Range = [500, 300]
K_Range_file = '{}_{}.csv'.format(K_Range[0], K_Range[1])
K_Range_dir = '{}_{}'.format(K_Range[0], K_Range[1])

#model_type = 'LSTM-LSTM'
model_type = 'TimeLinear'
#model_type = 'CNN-LSTM'
#model_type = 'ANN'
seq_length1 = 7
seq_length2 = seq_length1+1
min_contract_day = 2
forecast_variable = 'slope'

model_Dir_tree = ['Forecast&model', expiry, IV_type, 'K_{}'.format(K_Range_dir), model_type,\
                  'seq{}_seq{}_min{}'.format(seq_length1, seq_length2, min_contract_day), forecast_variable]
for model_dir in model_Dir_tree:
    if model_dir not in os.listdir(model_path):
        os.mkdir(model_path + model_dir)
    model_path = model_path + model_dir + '/'


file_path = model_path + 'model1.csv'
IV_data = pd.read_csv(file_path, index_col=False, encoding='Big5')


IV_matrix = np.array(IV_data)

columns_names = ['履約價', '隱含波動率({})'.format(s_c),'預測隱含波動率({})'.format(s_c), '期貨{}'.format(s_c)]
#columns_names = ['履約價(收盤價)', '隱含波動率(收盤價)','預測隱含波動率(價)']
column_index = [IV_data.columns.get_loc(col) for col in columns_names]

y = IV_matrix[:, column_index[1]]
y_hat = IV_matrix[:, column_index[2]]

L2_Distance = np.zeros(int(len(y_hat) / K_num))
L1_Distance = np.zeros(int(len(y_hat) / K_num))
L1_Distance_a = np.zeros(int(len(y_hat) / K_num))
for i in range(len(L2_Distance)):
    y_day = y[i*K_num:(i+1)*K_num]
    y_hat_day = y_hat[i*K_num:(i+1)*K_num]
    l1_distance = np.sum(abs(y_day - y_hat_day))
    l1_distance_a = np.sum(abs((y_day - y_hat_day) / y_day))
    l2_distance = np.sum((y_day - y_hat_day) * (y_day - y_hat_day))

    L2_Distance[i] = l2_distance
    L1_Distance[i] = l1_distance
    L1_Distance_a[i] = l1_distance_a

MSE = np.mean(L2_Distance)
MAE = np.mean(L1_Distance)
RMSE = np.sqrt(MSE)
MAPE = np.mean(L1_Distance_a)
print('MSE:{}'.format(MSE))
print('MAE:{}'.format(MAE))
print('RMSE:{}'.format(RMSE))
print('MAPE:{}%'.format(MAPE*100))





MSE:0.05335899997027592
MAE:1.6862015866771667
RMSE:0.2309956708907678
MAPE:1020.1731048534498%


In [62]:
import pandas as pd
import numpy as np
import os 




K_num = 100
s_c = '結算價'
expiry = 'NearbyMonth'

type_ = 'put'
IV_type = '{}IV'.format(type_)
if type_ == 'call':
    K_Range = [300, 500]
if type_ == 'put':
    K_Range = [500, 300]
K_Range_file = '{}_{}.csv'.format(K_Range[0], K_Range[1])
K_Range_dir = '{}_{}'.format(K_Range[0], K_Range[1])

#model_type = 'LSTM-LSTM'
#model_type = 'TimeLinear'
#model_type = 'CNN-LSTM'
model_type = 'ANN'
min_contract_day = 2
forecast_variable = 'slope'






Seq_Length1 = np.array([7, 15, 29])
Seq_Length2 = Seq_Length1 + 1
LossMatrix = np.zeros((len(Seq_Length1), 4))
for i in range(len(Seq_Length1)):
    model_path = './../../../'
    seq_length1 = Seq_Length1[i]
    seq_length2 = Seq_Length2[i]
    model_Dir_tree = ['Forecast&model', expiry, IV_type, 'K_{}'.format(K_Range_dir), model_type,\
                  'seq{}_seq{}_min{}'.format(seq_length1, seq_length2, min_contract_day), forecast_variable]
    for model_dir in model_Dir_tree:
        #if model_dir not in os.listdir(model_path):
            #os.mkdir(model_path + model_dir)
        model_path = model_path + model_dir + '/'


    file_path = model_path + 'model1.csv'
    IV_data = pd.read_csv(file_path, index_col=False, encoding='Big5')


    IV_matrix = np.array(IV_data)

    columns_names = ['履約價', '隱含波動率({})'.format(s_c),'預測隱含波動率({})'.format(s_c), '期貨{}'.format(s_c)]
    #columns_names = ['履約價(收盤價)', '隱含波動率(收盤價)','預測隱含波動率(價)']
    column_index = [IV_data.columns.get_loc(col) for col in columns_names]

    y = IV_matrix[:, column_index[1]]
    y_hat = IV_matrix[:, column_index[2]]

    L2_Distance = np.zeros(int(len(y_hat) / K_num))
    L1_Distance = np.zeros(int(len(y_hat) / K_num))
    L1_Distance_a = np.zeros(int(len(y_hat) / K_num))
    for j in range(len(L2_Distance)):
        y_day = y[j*K_num:(j+1)*K_num]
        y_hat_day = y_hat[j*K_num:(j+1)*K_num]
        l1_distance = np.sum(abs(y_day - y_hat_day))
        l1_distance_a = np.sum(abs((y_day - y_hat_day) / y_day))
        l2_distance = np.sum((y_day - y_hat_day) * (y_day - y_hat_day))

        L2_Distance[j] = l2_distance
        L1_Distance[j] = l1_distance
        L1_Distance_a[j] = l1_distance_a

    MSE = np.mean(L2_Distance)
    MAE = np.mean(L1_Distance)
    RMSE = np.sqrt(MSE)
    MAPE = np.mean(L1_Distance_a)
    LossMatrix[i] = np.array([MSE, MAE, RMSE, MAPE])



def format_number1(x):
        return "{:.6f}".format(x)
    
def format_number2(x):
    return "{:.2f}%".format(x*100)
    
LossTable = pd.DataFrame(data=LossMatrix, columns=['MSE', 'MAE', 'RMSE', 'MAPE'])
LossTable.iloc[:,:3] = LossTable.iloc[:,:3].applymap(format_number1)
LossTable.iloc[:,3] = LossTable.iloc[:,3:4].applymap(format_number2)
n_Data = pd.DataFrame({'n':[7, 15, 30]})
D = pd.concat([n_Data, LossTable], axis=1)
print(D)

model_path = './../../../'
model_Dir_tree = ['Forecast&model', 'comparison']
for model_dir in model_Dir_tree:
    model_path = model_path + model_dir + '/'
file_name = '{}_{}.csv'.format(model_type, type_)
D.to_csv(model_path + file_name, index=False, encoding='Big5')


    n       MSE       MAE      RMSE     MAPE
0   7  0.012831  0.872596  0.113274  538.30%
1  15  0.013146  0.670136  0.114657  403.76%
2  30  0.009308  0.692411  0.096475  420.76%


C:\Users\bgpa4\AppData\Local\Temp\ipykernel_11844\3105472307.py:90: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  LossTable.iloc[:,:3] = LossTable.iloc[:,:3].applymap(format_number1)
C:\Users\bgpa4\AppData\Local\Temp\ipykernel_11844\3105472307.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    0.012831
1    0.013146
2    0.009308
Name: MSE, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  LossTable.iloc[:,:3] = LossTable.iloc[:,:3].applymap(format_number1)
C:\Users\bgpa4\AppData\Local\Temp\ipykernel_11844\3105472307.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    0.872596
1    0.670136
2    0.692411
Name: MAE, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  LossTable.iloc[:,:3]